In [1]:
!nvidia-smi

Fri Apr 28 22:59:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install required packages
!pip -q install datasets
!pip -q install transformers
!pip -q install sentencepiece
!pip -q install hazm
!pip -q install clean-text[gpl]

In [3]:
# Import required packages
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm 

from hazm import Normalizer
from hazm import WordTokenizer
from cleantext import clean
import re
import csv

-  load train, test, and valid with Pandas

1.   List item
2.   List item



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_df = pd.read_csv('Project_Train_Data_FewShot_Learning_588samples.csv', delimiter=None, 
                header='infer', names=None, 
                index_col=None, usecols=None, squeeze=False, 
                engine='python', encoding='utf-8', error_bad_lines=False)
eval_df = pd.read_csv('Project_Val_Data_FewShot_Learning_588samples.csv', delimiter=None, 
                 header='infer', names=None, 
                 index_col=None, usecols=None, squeeze=False, 
                 engine='python', encoding='utf-8', error_bad_lines=False)
test_df = pd.read_csv('Project_Test_Data.csv', delimiter=None, 
                header='infer', names=None, 
                index_col=None, usecols=None, squeeze=False, 
                engine='python', encoding='utf-8', error_bad_lines=False)
train_df.sample(5)
# drop the label columns

train_df.drop(columns=['AGE', 'DIFFERENTIAL_DIAGNOSIS', 'SEX', 'PATHOLOGY', 'EVIDENCES', 'INITIAL_EVIDENCE', 'English Pathology'], inplace=True)
eval_df.drop(columns=['AGE', 'DIFFERENTIAL_DIAGNOSIS', 'SEX', 'PATHOLOGY', 'EVIDENCES', 'INITIAL_EVIDENCE', 'English Pathology'], inplace=True)
test_df.drop(columns=['AGE', 'DIFFERENTIAL_DIAGNOSIS', 'SEX', 'PATHOLOGY', 'EVIDENCES', 'INITIAL_EVIDENCE', 'English Pathology'], inplace=True)
train_df.head()

<ipython-input-5-030cb4213687>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train_df = pd.read_csv('Project_Train_Data_FewShot_Learning_588samples.csv', delimiter=None,
<ipython-input-5-030cb4213687>:1: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  train_df = pd.read_csv('Project_Train_Data_FewShot_Learning_588samples.csv', delimiter=None,
<ipython-input-5-030cb4213687>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  eval_df = pd.read_csv('Project_Val_Data_FewShot_Learning_588samples.csv', delimiter=None,
<ipython-input-5-030cb4213687>:5: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ev

,Unnamed: 0,OriginalRowKey,English Evidences,ICD-10
0,890172,890172,{'Do you have severe Chronic Obstructive Pulmo...,J44.1
1,519403,519403,{'Have you had one or several flare ups of chr...,J44.1
2,764990,764990,{'Do you have severe Chronic Obstructive Pulmo...,J44.1
3,162274,162274,{'Do you have severe Chronic Obstructive Pulmo...,J44.1
4,903972,903972,{'Are you experiencing shortness of breath or ...,J44.1


In [6]:
print(train_df.info())
print(eval_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         588 non-null    int64 
 1   OriginalRowKey     588 non-null    int64 
 2   English Evidences  588 non-null    object
 3   ICD-10             588 non-null    object
dtypes: int64(2), object(2)
memory usage: 18.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         588 non-null    int64 
 1   OriginalRowKey     588 non-null    int64 
 2   English Evidences  588 non-null    object
 3   ICD-10             588 non-null    object
dtypes: int64(2), object(2)
memory usage: 18.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Colu

In [7]:
icd_df = pd.read_excel('englishPathology2ICD.xlsx')
icd_df

,Pathology,ICD-10
0,acute copd exacerbation / infection,J44.1
1,acute dystonic reactions,G24.9
2,acute laryngitis,J04.0
3,acute otitis media,H66.9
4,acute pulmonary edema,J81.0
5,acute rhinosinusitis,J01.9
6,allergic sinusitis,J01
7,anaphylaxis,T78.0
8,anemia,D64.9
9,atrial fibrillation,I48.9


In [8]:
intlabel_2_icd10 = {}
icd10_2_intlabel = {}
for idx, row in enumerate(icd_df['ICD-10']):
    intlabel_2_icd10[idx] = row
    icd10_2_intlabel[row] = idx
print(intlabel_2_icd10)
print(icd10_2_intlabel)

{0: 'J44.1', 1: 'G24.9', 2: 'J04.0', 3: 'H66.9', 4: 'J81.0', 5: 'J01.9', 6: 'J01', 7: 'T78.0', 8: 'D64.9', 9: 'I48.9', 10: 'K22.3', 11: 'J47', 12: 'J21', 13: 'J20.9', 14: 'J98.0', 15: 'B57.5', 16: 'J32.9', 17: 'G44.0', 18: 'J05.0', 19: 'A98.4', 20: 'J05.1', 21: 'K21', 22: 'G61.0', 23: 'B20', 24: 'J10 or J11', 25: 'K40', 26: 'J38.5', 27: 'R60.0', 28: 'G70', 29: 'I51.4', 30: 'C25.9', 31: 'F41.0', 32: 'I30.9', 33: 'J18.9', 34: 'I21.4', 35: 'I47.1', 36: 'I26', 37: 'C34.9', 38: 'D86.9', 39: 'T61.1', 40: 'M32.9', 41: 'J93.1', 42: 'S22.3', 43: 'I20.9', 44: 'A15.9', 45: 'I20.0', 46: 'J06.9', 47: 'J02.9', 48: 'A37.9'}
{'J44.1': 0, 'G24.9': 1, 'J04.0': 2, 'H66.9': 3, 'J81.0': 4, 'J01.9': 5, 'J01': 6, 'T78.0': 7, 'D64.9': 8, 'I48.9': 9, 'K22.3': 10, 'J47': 11, 'J21': 12, 'J20.9': 13, 'J98.0': 14, 'B57.5': 15, 'J32.9': 16, 'G44.0': 17, 'J05.0': 18, 'A98.4': 19, 'J05.1': 20, 'K21': 21, 'G61.0': 22, 'B20': 23, 'J10 or J11': 24, 'K40': 25, 'J38.5': 26, 'R60.0': 27, 'G70': 28, 'I51.4': 29, 'C25.9': 30

## Normalization (Preprocessing)


In [9]:
# for more details see hazm doc
tokenizer = WordTokenizer()
normalizer = Normalizer()

def cleaning(x):
    x = normalizer.normalize(x)
    return x.strip()

def text_preprocessor(t, tokenize=False):
    tokens = tokenizer.tokenize(cleaning(normalizer.normalize(t)))
    return tokens if tokenize else ' '.join(tokens)

In [10]:
# clean test set and valid set
test_df = test_df.rename(columns={'English Evidences': 'content'})
eval_df = eval_df.rename(columns={'English Evidences': 'content'})
train_df = train_df.rename(columns={'English Evidences': 'content'})
test_df['cleaned'] = test_df.content.apply(text_preprocessor)
print('test cleaned')
eval_df['cleaned'] = eval_df.content.apply(text_preprocessor)
print('valid cleaned')

test cleaned
valid cleaned


In [11]:
# clean and tokenize train set
train_df['cleaned'] = train_df.content.apply(text_preprocessor)
print('train cleaned')
train_df['tokens'] = train_df.content.apply(text_preprocessor, tokenize=True)
train_df.head(3)

train cleaned


,Unnamed: 0,OriginalRowKey,content,ICD-10,cleaned,tokens
0,890172,890172,{'Do you have severe Chronic Obstructive Pulmo...,J44.1,{ 'Do you have severe Chronic Obstructive Pulm...,"[{, 'Do, you, have, severe, Chronic, Obstructi..."
1,519403,519403,{'Have you had one or several flare ups of chr...,J44.1,{ 'Have you had one or several flare ups of ch...,"[{, 'Have, you, had, one, or, several, flare, ..."
2,764990,764990,{'Do you have severe Chronic Obstructive Pulmo...,J44.1,{ 'Do you have severe Chronic Obstructive Pulm...,"[{, 'Do, you, have, severe, Chronic, Obstructi..."


### Finding the max len

In [12]:
# compute the words len in each magazine
train_df['words_len'] = train_df.tokens.apply(lambda t: len(t))

In [13]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=train_df['words_len']))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [14]:
minlim, maxlim = 30, 2048
# remove comments with the length of fewer than 30 words
print('size of the data before remove: ', len(train_df))
train_df['words_len'] = train_df['words_len'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
train_df = train_df.dropna(subset=['words_len'])
train_df = train_df.reset_index(drop=True)
print('size of the data after remove: ', len(train_df))

size of the data before remove:  588
size of the data after remove:  588


### Distribution of Classes

In [15]:
test_df = test_df.rename(columns={'ICD-10': 'label_id'})
test_df.sample(5)
#test_df['label_id'] = test_df['label_id'].to_numeric(int)
#pd.to_numeric(test_df['label_id'])
test_df.sample(5)
test_df['label_id'].value_counts()

J06.9         652
J02.9         620
D64.9         505
B20           289
R60.0         275
J18.9         274
T78.0         273
H66.9         272
J10 or J11    264
I26           259
K21           254
G24.9         253
F41.0         248
J32.9         243
J20.9         240
I21.4         237
J04.0         230
I30.9         229
D86.9         224
I20.0         219
J81.0         206
G61.0         205
I20.9         201
G44.0         195
T61.1         195
J05.1         194
I48.9         187
I47.1         179
K40           179
J47           175
J98.0         173
J01           172
C25.9         168
K22.3         162
J44.1         160
G70           159
C34.9         155
J01.9         148
A15.9         148
I51.4         111
M32.9         101
J93.1          95
B57.5          79
J38.5          58
A37.9          51
S22.3          49
J05.0          26
J21             5
A98.4           4
Name: label_id, dtype: int64

In [16]:
print('Total number of intents: %d'%(len(test_df['label_id'].value_counts().index)))

Total number of intents: 49


In [17]:
# test_map_dict = {k: v for v, k in enumerate(test_df.label_id.unique())}
test_df['label_id'] = test_df['label_id'].map(icd10_2_intlabel)
eval_df = eval_df.rename(columns={'ICD-10': 'label_id'})
eval_df['label_id'] = eval_df['label_id'].map(icd10_2_intlabel)

In [18]:
icd10_2_intlabel.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48])

In [19]:
test_df['tokens'] = test_df.content.apply(text_preprocessor, tokenize=True)
eval_df['tokens'] = eval_df.content.apply(text_preprocessor, tokenize=True)

In [20]:
test_df

,OriginalRowKey,content,label_id,cleaned,tokens
0,109830,{'Are you experiencing shortness of breath or ...,14,{ 'Are you experiencing shortness of breath or...,"[{, 'Are, you, experiencing, shortness, of, br..."
1,11402,{'Do you regularly take stimulant drugs?': 'Tr...,1,{ 'Do you regularly take stimulant drugs ? ' :...,"[{, 'Do, you, regularly, take, stimulant, drug..."
2,30840,"{'Do you have pain somewhere, related to your ...",16,"{ 'Do you have pain somewhere, related to your...","[{, 'Do, you, have, pain, somewhere,, related,..."
3,14889,{'Have you ever had a sexually transmitted inf...,23,{ 'Have you ever had a sexually transmitted in...,"[{, 'Have, you, ever, had, a, sexually, transm..."
4,131903,"{'Do you feel anxious?': 'True', 'Do any membe...",31,"{ 'Do you feel anxious ? ' : 'True', 'Do any m...","[{, 'Do, you, feel, anxious, ?, ', :, 'True',,..."
...,...,...,...,...,...
9995,113311,{'Have you ever had a sexually transmitted inf...,23,{ 'Have you ever had a sexually transmitted in...,"[{, 'Have, you, ever, had, a, sexually, transm..."
9996,46200,{'Are you infected with the human immunodefici...,34,{ 'Are you infected with the human immunodefic...,"[{, 'Are, you, infected, with, the, human, imm..."
9997,54048,"{'Have you been coughing up blood?': 'True', '...",47,{ 'Have you been coughing up blood ? ' : 'True...,"[{, 'Have, you, been, coughing, up, blood, ?, ..."
9998,27233,{'Have you lost consciousness associated with ...,38,{ 'Have you lost consciousness associated with...,"[{, 'Have, you, lost, consciousness, associate..."


In [21]:
eval_df

,Unnamed: 0,OriginalRowKey,content,label_id,cleaned,tokens
0,26182,26182,{'Do you have severe Chronic Obstructive Pulmo...,0,{ 'Do you have severe Chronic Obstructive Pulm...,"[{, 'Do, you, have, severe, Chronic, Obstructi..."
1,129844,129844,{'Are you experiencing shortness of breath or ...,0,{ 'Are you experiencing shortness of breath or...,"[{, 'Are, you, experiencing, shortness, of, br..."
2,14406,14406,{'Do you have severe Chronic Obstructive Pulmo...,0,{ 'Do you have severe Chronic Obstructive Pulm...,"[{, 'Do, you, have, severe, Chronic, Obstructi..."
3,38411,38411,{'Do you have severe Chronic Obstructive Pulmo...,0,{ 'Do you have severe Chronic Obstructive Pulm...,"[{, 'Do, you, have, severe, Chronic, Obstructi..."
4,98519,98519,{'Do you have severe Chronic Obstructive Pulmo...,0,{ 'Do you have severe Chronic Obstructive Pulm...,"[{, 'Do, you, have, severe, Chronic, Obstructi..."
...,...,...,...,...,...,...
583,6497,6497,{'Have you been in contact with someone who ha...,48,{ 'Have you been in contact with someone who h...,"[{, 'Have, you, been, in, contact, with, someo..."
584,87059,87059,{'Are you significantly overweight compared to...,48,{ 'Are you significantly overweight compared t...,"[{, 'Are, you, significantly, overweight, comp..."
585,28040,28040,{'Are you significantly overweight compared to...,48,{ 'Are you significantly overweight compared t...,"[{, 'Are, you, significantly, overweight, comp..."
586,119223,119223,{'Are you significantly overweight compared to...,48,{ 'Are you significantly overweight compared t...,"[{, 'Are, you, significantly, overweight, comp..."


In [22]:
# eval_df = eval_df.rename(columns={'ICD-10': 'label_id'})
# eval_map_dict = {k: v for v, k in enumerate(eval_df.label_id.unique())}
# eval_df['label_id'] = eval_df['label_id'].map(eval_map_dict)
train_df = train_df.rename(columns={'ICD-10': 'label_id'})
# train_map_dict = {k: v for v, k in enumerate(train_df.label_id.unique())}
train_df['label_id'] = train_df['label_id'].map(icd10_2_intlabel)
fig = go.Figure()
topic_freq = train_df.label_id.value_counts()
fig.add_trace(go.Bar(y=topic_freq, x=topic_freq.index.to_numpy()))
fig.update_layout(
    title_text='Distribution of each class',
    xaxis_title_text='classes',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

## Prepare data for Text Classification
- Using transformers tokenizer
- This tokenizer removes half-space and newlines in its tokenizer (you don't have to handle it in your cleaning)

In [23]:
from datasets import load_metric
from transformers import AutoTokenizer

In [24]:
lr = 2e-5
weight_decay = 0.001
num_epochs = 12
batch_size = 4 #DO NOT INCREASE >= 20, causes CUDA memory issue. If memory issue occurs, delete runtime and decrease batch size
accumulation_steps = 4
rs = 2
num_workers = 2
max_len = 1024 # much bigger than bert :) 
class_number = 49 #DO NOT CHANGE
metric = load_metric('f1')
model_name = "SajjadAyoubi/distil-bigbird-fa-zwnj"
max_len = 1024 # much bigger than bert :) 
class_number = 49 #DO NOT CHANGE
metric = load_metric('f1')
model_name = "SajjadAyoubi/distil-bigbird-fa-zwnj"

<ipython-input-24-0e7b93f066b3>:10: FutureWarning:

load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate



In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

- Tokenization

In [26]:
start = 0
end = 80
batches = []
while end <= 10000:
  test_dfBatch = test_df.iloc[start:end]
  print(test_dfBatch.head())
  start += 80
  end += 80
  batches.append(test_dfBatch)

   OriginalRowKey                                            content  \
0          109830  {'Are you experiencing shortness of breath or ...   
1           11402  {'Do you regularly take stimulant drugs?': 'Tr...   
2           30840  {'Do you have pain somewhere, related to your ...   
3           14889  {'Have you ever had a sexually transmitted inf...   
4          131903  {'Do you feel anxious?': 'True', 'Do any membe...   

   label_id                                            cleaned  \
0        14  { 'Are you experiencing shortness of breath or...   
1         1  { 'Do you regularly take stimulant drugs ? ' :...   
2        16  { 'Do you have pain somewhere, related to your...   
3        23  { 'Have you ever had a sexually transmitted in...   
4        31  { 'Do you feel anxious ? ' : 'True', 'Do any m...   

                                              tokens  
0  [{, 'Are, you, experiencing, shortness, of, br...  
1  [{, 'Do, you, regularly, take, stimulant, drug...  
2  [{

In [27]:
train_encodings = tokenizer(train_df.cleaned.tolist(), max_length=max_len, truncation=True, padding=True)
print('train tokenized')
allEncodings = []
for b in batches:
  allEncodings.append(tokenizer(b.cleaned.tolist(), max_length=max_len, truncation=True, padding=True))
print(len(allEncodings))
print('test tokenized')
valid_encodings = tokenizer(eval_df.cleaned.tolist(), max_length=max_len, truncation=True, padding=True)
print('valid tokenized')

train tokenized
125
test tokenized
valid tokenized


- Create config for our classification task

In [28]:
from transformers import AutoConfig
# create a dict for classes
label2id = icd10_2_intlabel
id2label = intlabel_2_icd10
config = AutoConfig.from_pretrained(model_name, **{'label2id': label2id, 'id2label': id2label})
print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'J44.1': 0, 'G24.9': 1, 'J04.0': 2, 'H66.9': 3, 'J81.0': 4, 'J01.9': 5, 'J01': 6, 'T78.0': 7, 'D64.9': 8, 'I48.9': 9, 'K22.3': 10, 'J47': 11, 'J21': 12, 'J20.9': 13, 'J98.0': 14, 'B57.5': 15, 'J32.9': 16, 'G44.0': 17, 'J05.0': 18, 'A98.4': 19, 'J05.1': 20, 'K21': 21, 'G61.0': 22, 'B20': 23, 'J10 or J11': 24, 'K40': 25, 'J38.5': 26, 'R60.0': 27, 'G70': 28, 'I51.4': 29, 'C25.9': 30, 'F41.0': 31, 'I30.9': 32, 'J18.9': 33, 'I21.4': 34, 'I47.1': 35, 'I26': 36, 'C34.9': 37, 'D86.9': 38, 'T61.1': 39, 'M32.9': 40, 'J93.1': 41, 'S22.3': 42, 'I20.9': 43, 'A15.9': 44, 'I20.0': 45, 'J06.9': 46, 'J02.9': 47, 'A37.9': 48}
id2label: {0: 'J44.1', 1: 'G24.9', 2: 'J04.0', 3: 'H66.9', 4: 'J81.0', 5: 'J01.9', 6: 'J01', 7: 'T78.0', 8: 'D64.9', 9: 'I48.9', 10: 'K22.3', 11: 'J47', 12: 'J21', 13: 'J20.9', 14: 'J98.0', 15: 'B57.5', 16: 'J32.9', 17: 'G44.0', 18: 'J05.0', 19: 'A98.4', 20: 'J05.1', 21: 'K21', 22: 'G61.0', 23: 'B20', 24: 'J10 or J11', 25: 'K40', 26: 'J38.5', 27: 'R60.0', 28: 'G70', 29: 

# Training

In [29]:
import torch
from torch import optim
from torch.utils.data import DataLoader
from transformers import BigBirdForSequenceClassification
from transformers import default_data_collator

In [30]:
train_df.label_id


0       0
1       0
2       0
3       0
4       0
       ..
583    48
584    48
585    48
586    48
587    48
Name: label_id, Length: 588, dtype: int64

In [31]:
batches[0].label_id

0     14
1      1
2     16
3     23
4     31
      ..
75    37
76     4
77    37
78    46
79    40
Name: label_id, Length: 80, dtype: int64

In [32]:
class DigiMagDs(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_encodings = tokenizer(test_df.cleaned.tolist(), max_length=max_len, truncation=True, padding=True)
# create datasets
train_ds = DigiMagDs(train_encodings, train_df.label_id)
test_ds = DigiMagDs(test_encodings, test_df.label_id)
valid_ds = DigiMagDs(valid_encodings, eval_df.label_id)
digiMagDsBatches = []
for i in range(len(batches)):
  updated = batches[i].reset_index(drop=True)
  temp = DigiMagDs(allEncodings[i], updated.label_id)
  digiMagDsBatches.append(temp)

# create Dataloders
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                      num_workers=num_workers, collate_fn=default_data_collator)

valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=True,
                       num_workers=num_workers, collate_fn=default_data_collator)

testDLDataLoaderObjects = []
for i in range(len(batches)):
  curr_batch = DataLoader(digiMagDsBatches[i], batch_size=batch_size, shuffle=True,
                     num_workers=num_workers, collate_fn=default_data_collator)
  testDLDataLoaderObjects.append(curr_batch)

print(len(testDLDataLoaderObjects))

125


## PyTorch

In [33]:
import time, sys
from IPython.display import clear_output

class TorchTrainer:
    def __init__(self, model, train_dl, valid_dl, test_dl_batches, optimizer, device, accumulation_steps, compute_metric):
        self.model = model.to(device)
        self.device = device
        self.train_dl = train_dl
        self.valid_dl = valid_dl
        self.test_dl_batches = test_dl_batches
        self.optimizer = optimizer
        self.compute_metric = compute_metric
        self.accumulation_steps = accumulation_steps
        self.loss_history = []
        self.outputList = []

    def predictLabels(self, batchNumber):
        data = self.test_dl_batches[batchNumber]

        if (batchNumber % 10 == 0):
              print("batch " + str(batchNumber))
                
        #print(data)
        for i, batch in enumerate(data):
              #print(batch)
              input_ids = batch['input_ids'].to(self.device)
              attention_mask = batch['attention_mask'].to(self.device)
              labels = batch['labels'].to(self.device)
              #print(labels)
              outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
              predictions = torch.argmax(outputs['logits'], dim=1)
              

              self.outputList.append(predictions)
              del labels
              del attention_mask
              del input_ids


    def fit(self, num_epochs):
        clear_output()
        valid_acc = 0
        for epoch in range(num_epochs):
            print('Epoch %2d/%2d' % (epoch + 1, num_epochs))
            #print(1)
            print('-' * 20)
            t0 = time.time()
            #print(2)
            train_acc = self.train_model()
            #print(train_acc)
            valid_acc = self.valid_model()
            #print(valid_acc)
            time_elapsed = time.time() - t0
            print('\n  Metrics: | train_met: %.3f | valid_met: %.3f |' % (train_acc[0], valid_acc[0]))
            print('\n  Epoch complete in: %.0fm %.0fs \n' % (time_elapsed // 60, time_elapsed % 60))
        return

    def train_model(self):
        self.model.train()
        #print(1)
        N = len(self.train_dl.dataset)
        #print(2)
        steps = N // (self.train_dl.batch_size*self.accumulation_steps) 
        #print(3)
        step = 0
        avg_metric = 0.0
        it = iter(self.train_dl)
        first = next(it)
        #print(first)
        for i, batch in enumerate(self.train_dl):
            # print("error3")
            # print(i)
            # print(batch)
            #print("for loop")
            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            labels = batch['labels'].to(self.device)
            outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
            predictions = torch.argmax(outputs['logits'], dim=1)
            loss = outputs['loss'] 
            loss /= self.accumulation_steps   
            loss.backward()
            # weights update
            if ((i+1) % self.accumulation_steps == 0) or (step == steps):
                self.optimizer.step()
                self.optimizer.zero_grad()
           
                avg_metric += self.compute_metric(predictions, labels)
                self.loss_history.append(loss)
                sys.stdout.flush()
                print(f'"\r  Train_Step: {step}/{steps} | running_loss: {loss}')
                #sys.stdout.write("\r  Train_Step: %d/%d | runing_loss: %.4f" % (step, steps, loss))
                step += 1
            del labels
            del attention_mask
            del input_ids

        sys.stdout.flush()
        return torch.tensor([avg_metric]) / step

    def valid_model(self):
        #print()
        self.model.eval()
        N = len(self.valid_dl.dataset)
        steps = N // self.valid_dl.batch_size
        avg_metric = 0.0
        with torch.no_grad():
            for i, batch in enumerate(self.valid_dl):
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                #print(labels)
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                predictions = torch.argmax(outputs['logits'], dim=1)
                if i == 0:
                  print('')
                  print('labels are: ' + str(labels))
                  print('predictions are ' + str(predictions))
                
                loss = outputs['loss']
                avg_metric += self.compute_metric(predictions, labels)
                sys.stdout.flush()
                sys.stdout.write("\r  Valid_Step: %d/%d | runing_loss: %.4f" % (i, steps, loss))
                del labels
                del attention_mask
                del input_ids

        sys.stdout.flush()
        return torch.tensor([avg_metric]) / steps

In [34]:
model = BigBirdForSequenceClassification.from_pretrained(model_name, config=config)
# model.gradient_checkpointing_enable() # for avoiding OOM

optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Some weights of the model checkpoint at SajjadAyoubi/distil-bigbird-fa-zwnj were not used when initializing BigBirdForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint 

In [35]:
def compute_metric(predictions, labels):
    return metric.compute(predictions=predictions, 
                          references=labels, average="weighted")['f1']

In [36]:
trainer = TorchTrainer(model, train_dl, valid_dl, testDLDataLoaderObjects, optimizer=optimizer, 
                       device=device, accumulation_steps=accumulation_steps, 
                       compute_metric=compute_metric)
trainer.fit(num_epochs=num_epochs)

Epoch  1/12
--------------------
  Train_Step: 0/36 | running_loss: 0.98555588722229
  Train_Step: 1/36 | running_loss: 0.924805760383606
  Train_Step: 2/36 | running_loss: 0.9676297903060913
  Train_Step: 3/36 | running_loss: 0.9251912236213684
  Train_Step: 4/36 | running_loss: 1.0486373901367188
  Train_Step: 5/36 | running_loss: 0.9930902719497681
  Train_Step: 6/36 | running_loss: 1.0048567056655884
  Train_Step: 7/36 | running_loss: 1.0196435451507568
  Train_Step: 8/36 | running_loss: 0.9961071610450745
  Train_Step: 9/36 | running_loss: 1.0339560508728027
  Train_Step: 10/36 | running_loss: 0.9764376878738403
  Train_Step: 11/36 | running_loss: 1.0223122835159302
  Train_Step: 12/36 | running_loss: 0.9518158435821533
  Train_Step: 13/36 | running_loss: 0.9593717455863953
  Train_Step: 14/36 | running_loss: 1.0159999132156372
  Train_Step: 15/36 | running_loss: 0.9497250914573669
  Train_Step: 16/36 | running_loss: 1.0045212507247925
  Train_Step: 17/36 | running_loss: 1.0124770

In [37]:
torch.cuda.empty_cache()

In [38]:
torch.save(trainer, '/content/drive/MyDrive/BigBird.pt')

In [39]:
!nvidia-smi

Fri Apr 28 23:24:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    32W /  70W |   2405MiB / 15360MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [40]:
print(len(batches))

125


In [41]:
torch.cuda.empty_cache()
#trainer.predictLabels(1)
for i in range(len(batches)):
    torch.cuda.empty_cache()
    trainer.predictLabels(i)
    torch.cuda.empty_cache()

batch 0
batch 10
batch 20
batch 30
batch 40
batch 50
batch 60
batch 70
batch 80
batch 90
batch 100
batch 110
batch 120


In [42]:
import tensorflow as tf

In [43]:
output = trainer.outputList
flattened = []
for tens in output:
  for val in tens:
    flattened.append(val.item())
header = [['Predicted Values']]
#Be sure to change filepath below so you don't override existing file
file = open('/content/drive/MyDrive/predictedBigBird.csv', 'w+', newline ='') 
ICD10Codes = []
for val in flattened:
  ICD10Codes.append(id2label[val])

print(len(ICD10Codes))
dataToWrite = map(lambda x: [x], ICD10Codes)
# writing the data into the file
with file:   
    write = csv.writer(file)
    write.writerows(header)
    write.writerows(dataToWrite)

10000


## HuggingFace 🤗
- You can also train it with HuggingFace Trainer [more details](https://huggingface.co/transformers/main_classes/trainer.html)

In [44]:
from transformers import TrainingArguments, Trainer

In [45]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [46]:
model = BigBirdForSequenceClassification.from_pretrained(model_name, config=config)

Some weights of the model checkpoint at SajjadAyoubi/distil-bigbird-fa-zwnj were not used when initializing BigBirdForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint 

In [47]:
args = TrainingArguments(
    "ParsBigBirdDigiMag",
    evaluation_strategy="epoch",
    logging_strategy='epoch',
    save_strategy="no",
    learning_rate=lr,
    label_smoothing_factor=0.05,
    weight_decay=weight_decay,
    fp16=True,
    dataloader_num_workers=num_workers,
    gradient_accumulation_steps=accumulation_steps,
    gradient_checkpointing=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    report_to='none',
    )

In [48]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=train_ds,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics
    )

In [49]:

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



Epoch,Training Loss,Validation Loss,F1
0,3.999600,3.819995,0.077124
1,3.759600,3.529383,0.352948
2,3.437300,3.123996,0.614170
4,3.097300,2.752588,0.712213
4,2.884200,2.446858,0.760828
5,2.556000,2.189481,0.797441
6,2.341900,1.972698,0.846074
8,2.157900,1.814275,0.851356
8,2.086000,1.693924,0.865648
9,1.930600,1.606734,0.879335


TrainOutput(global_step=432, training_loss=2.676692997967755, metrics={'train_runtime': 1144.9786, 'train_samples_per_second': 6.163, 'train_steps_per_second': 0.377, 'total_flos': 1407898224562176.0, 'train_loss': 2.676692997967755, 'epoch': 11.76})

In [50]:
trainer.evaluate(test_ds)

{'eval_loss': 1.8629175424575806,
 'eval_f1': 0.7659862802249582,
 'eval_runtime': 465.7241,
 'eval_samples_per_second': 21.472,
 'eval_steps_per_second': 5.368,
 'epoch': 11.76}